In [2]:
import pandas as pd
import scipy
import datetime
import calendar



In [3]:
def samedis_annee(annee: int):
    samedis = []
    for mois in range(1, 13):
        nb_jours = calendar.monthrange(annee, mois)[1]
        for jour in range(1, nb_jours + 1):
            date = datetime.date(annee, mois, jour)
            if date.weekday() == 5:  
                samedis.append([jour, mois])
    return samedis

Sam_data = pd.DataFrame(samedis_annee(2023),columns=['jour','mois'])


In [4]:
Sam_data

,jour,mois
0,7,1
1,14,1
2,21,1
3,28,1
4,4,2
5,11,2
6,18,2
7,25,2
8,4,3
9,11,3


In [5]:
Lieux = pd.read_csv(r'dataset_base\lieux-2023.csv', sep=';',low_memory=False)

In [6]:
Caract = pd.read_csv(r'dataset_base\caract-2023.csv', sep=';',low_memory=False)

In [7]:
Caract

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202300000001,7,5,2023,06:00,1,75,75101,2,4,2,7,RUE DE RIVOLI,"48,86638600","2,32347100"
1,202300000002,7,5,2023,05:30,5,94,94080,2,1,3,6,Avenue de Paris,"48,84547782","2,42868146"
2,202300000003,7,5,2023,20:50,1,94,94022,2,3,2,1,Avenue du Général Leclerc,"48,76240000","2,40655000"
3,202300000004,6,5,2023,23:57,5,94,94078,2,1,3,5,Rue de Paris,"48,73248432","2,44687575"
4,202300000005,7,5,2023,00:50,5,94,94068,2,2,3,3,56bis Avenue Raspail,"48,78581000","2,49217000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54817,202300054818,26,10,2023,20:45,5,974,97416,2,1,1,6,LA FONTAINE (RUE JEAN DE),"-21,33828000","55,47771000"
54818,202300054819,26,10,2023,19:10,3,974,97416,1,1,1,3,RN3 (ANCIENNE ROUTE),"-21,28865000","55,50994000"
54819,202300054820,26,10,2023,09:40,1,974,97411,2,1,1,7,BAMBOU (CHEMIN DE LA RUELLE),"-20,90129000","55,40598000"
54820,202300054821,26,10,2023,17:20,1,973,97302,1,6,1,3,ROUTE NATIONALE 1,"4,89713000","-52,32854000"


In [8]:
Lieux 

,Num_Acc,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,202300000001,4,RUE DE RIVOLI,0,NaN,1,2,0,1,-1,-1,1,NaN,-1,2,0,1,30
1,202300000001,4,RUE SAINT FLORENTIN,0,NaN,1,1,0,1,-1,-1,1,NaN,-1,2,0,1,30
2,202300000002,3,120,0,NaN,2,3,2,1,-1,-1,1,NaN,-1,2,0,1,50
3,202300000003,3,5,0,NaN,2,4,0,1,1,0,1,NaN,-1,2,5,1,50
4,202300000003,3,87,0,NaN,2,4,0,1,1,0,1,NaN,-1,2,5,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70855,202300054818,4,LA FONTAINE (RUE JEAN DE),0,NaN,2,2,0,1,-1,0,1,NaN,-1,1,0,1,50
70856,202300054819,4,RN3 (ANCIENNE ROUTE),0,NaN,2,2,0,2,0,0,1,NaN,-1,1,0,1,50
70857,202300054820,4,BAMBOU (CHEMIN DE LA RUELLE),0,NaN,2,2,0,2,0,0,2,NaN,-1,1,0,8,30
70858,202300054821,2,ROUTE NATIONALE 1,0,NaN,-1,2,1,1,4,50,1,NaN,-1,1,0,1,90


In [9]:
Samedi=Caract.merge(Sam_data, on=["jour","mois"], how="inner").sort_values(by=["mois","jour"]).reset_index(drop=True)

In [10]:
Samedi

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202300005200,7,1,2023,16:04,1,44,44109,2,1,1,6,1 Boulevard Général de Gaulle,"47,20738000","-1,54007238"
1,202300011274,7,1,2023,13:20,1,74,74010,2,1,1,1,NANFRAY (ROUTE DE),"45,89410000","6,08803000"
2,202300019151,7,1,2023,19:30,3,13,13077,1,1,1,4,RN 568,"43,41367350","5,00434666"
3,202300021478,7,1,2023,16:56,1,36,36044,2,1,1,7,avenue jacques chirac,"46,81514000","1,68482000"
4,202300031190,7,1,2023,17:27,2,92,92073,2,1,2,2,JAURES (AVENUE JEAN),"48,86453390","2,20746696"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7756,202300010831,30,12,2023,03:05,5,38,38382,1,1,8,6,a.48 pk 090.500 direction lyon,"45,24200400","5,65903200"
7757,202300010850,30,12,2023,11:15,1,973,97305,1,1,1,3,Route Nationale 1,"4,99159500","-52,44990000"
7758,202300010866,30,12,2023,03:15,3,77,77153,1,1,2,5,N2,"49,05439000","2,69953100"
7759,202300010874,30,12,2023,05:40,3,04,04106,1,3,1,3,D4096,"43,96116200","5,89991000"


In [11]:
Samedi_lieux = Samedi.merge(Lieux, on="Num_Acc", how="inner")

In [12]:
Samedi_lieux_nuits= Samedi_lieux[Samedi_lieux["lum"].isin([3, 4, 5])]

In [13]:
Samedi_lieux[Samedi_lieux["lum"].isin([3, 4, 5])]["catr"].map({
    1: "Autoroute",
    2: "Route nationale",
    3: "Route départementale",
    4: "Voie communale",
    5: "Hors réseau public",
    6: "Parc de stationnement ouvert à la circulation publique",
    7: "Routes de métropole urbaine",
    9: "Autre"
}).value_counts(normalize=True) * 100


catr
Voie communale                                            42.350204
Route départementale                                      37.870855
Autoroute                                                  8.958697
Route nationale                                            7.591623
Routes de métropole urbaine                                2.472368
Parc de stationnement ouvert à la circulation publique     0.494474
Autre                                                      0.261780
Name: proportion, dtype: float64

##1 Relation age / accidents
##2 Sans permis

In [15]:
usagers = pd.read_csv(r'dataset_base\usagers-2023.csv', sep=';', low_memory=False)
vehicules = pd.read_csv(r'dataset_base\vehicules-2023.csv', sep=';', low_memory=False)
caract = pd.read_csv(r'dataset_base\caract-2023.csv', sep=';', low_memory=False)
lieux = pd.read_csv(r'dataset_base\lieux-2023.csv', sep=';', low_memory=False)

merged_data = pd.merge(
    usagers,
    vehicules,
    on=['Num_Acc', 'id_vehicule', 'num_veh'],
    suffixes=('_usager', '_vehicule')
)

merged_data = pd.merge(merged_data, caract, on='Num_Acc')
merged_data = pd.merge(merged_data, lieux, on='Num_Acc')


In [18]:
merged_data.drop_duplicates().to_csv('merged_data2023.csv', index=False)

In [19]:
f2023=pd.read_csv('merged_data2023.csv')

C:\Users\moham\AppData\Local\Temp\ipykernel_49956\2276383783.py:1: DtypeWarning: Columns (46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  f2023=pd.read_csv('merged_data2023.csv')


In [23]:
f2023['age'] = f2023['an'] - f2023['an_nais']
accidents_cibles = f2023.query("catu == 1 and age >= 20 and age <= 25")['Num_Acc'].unique()

result20_25 = f2023[f2023['Num_Acc'].isin(accidents_cibles)].reset_index()

In [27]:
result20_25

,index,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,...,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma,age
0,13,202300000005,203 851 173,155 680 549,B01,1,1,4,2,2001.0,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
1,14,202300000005,203 851 173,155 680 549,B01,1,1,4,2,2001.0,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
2,15,202300000005,203 851 174,155 680 550,A01,1,1,1,1,2001.0,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
3,16,202300000005,203 851 174,155 680 550,A01,1,1,1,1,2001.0,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
4,29,202300000009,203 851 162,155 680 541,A01,1,1,1,1,1988.0,...,-1,-1,1,NaN,-1,2,5,1,50,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46692,163674,202300054819,203 720 724,155 583 349,A01,1,1,3,1,2003.0,...,0,0,1,NaN,-1,1,0,1,50,20.0
46693,163675,202300054819,203 720 725,155 583 350,B01,1,1,1,2,1982.0,...,0,0,1,NaN,-1,1,0,1,50,41.0
46694,163680,202300054822,203 720 717,155 583 344,A01,2,2,1,2,2003.0,...,-1,-1,1,NaN,-1,1,0,5,30,20.0
46695,163681,202300054822,203 720 718,155 583 344,A01,1,1,1,2,2002.0,...,-1,-1,1,NaN,-1,1,0,5,30,21.0


In [ ]:
result20_25.reset_index().to_csv('result20_25.csv', index=False?)